# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [14]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [15]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [16]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'NEEQST OBAAK) E/S(CAT LRDLNMLGGAS',
 'calculationPrice': 'close',
 'open': 136.72,
 'openTime': 1662681046419,
 'openSource': 'ilifcafo',
 'close': 143.38,
 'closeTime': 1689089704019,
 'closeSource': 'fiocalif',
 'high': 143.6,
 'highTime': 1658569242135,
 'highSource': ' cdlume5 ii nedeaypre1t',
 'low': 138.06,
 'lowTime': 1662834680170,
 'lowSource': 'delyiaecpt 5nr m1 udeie',
 'latestPrice': 141.12,
 'latestSource': 'Close',
 'latestTime': 'February 4, 2021',
 'latestUpdate': 1689665268542,
 'latestVolume': 88381768,
 'iexRealtimePrice': 139.03,
 'iexRealtimeSize': 102,
 'iexLastUpdated': 1668731045116,
 'delayedPrice': 143.94,
 'delayedPriceTime': 1625688802457,
 'oddLotDelayedPrice': 142.56,
 'oddLotDelayedPriceTime': 1684701001295,
 'extendedPrice': 140.45,
 'extendedChange': -0.44,
 'extendedChangePercent': -0.00323,
 'extendedPriceTime': 1622897066266,
 'previousClose': 138.09,
 'previousVolume': 92444239,
 '

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [17]:
price = data['latestPrice']
market_cap = data['marketCap']
symbol = data['symbol']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [18]:
columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns=columns)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [19]:
df.append(pd.Series([symbol,price,market_cap,'N/A'],index=columns),ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,141.12,2328826725287,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [20]:
# df = pd.DataFrame(columns=columns)
# for stock in stocks['Ticker']:
#     api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(api_url).json()
#     df = df.append(pd.Series([stock,data['latestPrice'],data['marketCap'],'N/A'],index=columns),ignore_index=True)


In [21]:
len(df)

0

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [22]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [23]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
df = pd.DataFrame(columns=columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(pd.Series([symbol,data[symbol]['quote']['latestPrice'],data[symbol]['quote']['marketCap'],'N/A'],index=columns),ignore_index=True)
df      

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.01,38201363214,N/A
1,AAL,17.50,10813281388,N/A
2,AAP,164.27,10876759303,N/A
3,AAPL,141.40,2397046436792,N/A
4,ABBV,110.25,198875800855,N/A
...,...,...,...,...
500,YUM,107.39,31397776425,N/A
501,ZBH,168.36,34584536315,N/A
502,ZBRA,420.07,22408332477,N/A
503,ZION,50.19,8286309386,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [24]:
portfolio_size = input('Enter the vale of your portfolio:')

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print('Invalid value. Enter a number value')

Enter the vale of your portfolio:10000000
10000000.0


In [25]:
position_size = val/len(df.index)
for i in range(0, len(df.index)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / df.loc[i, 'Stock Price'])

df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.01,38201363214,152
1,AAL,17.50,10813281388,1131
2,AAP,164.27,10876759303,120
3,AAPL,141.40,2397046436792,140
4,ABBV,110.25,198875800855,179
...,...,...,...,...
500,YUM,107.39,31397776425,184
501,ZBH,168.36,34584536315,117
502,ZBRA,420.07,22408332477,47
503,ZION,50.19,8286309386,394


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [26]:
writer = pd.ExcelWriter('trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [27]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({'font_color':font_color,'bg_color':background_color,'border':1})
dollar_format = writer.book.add_format({'font_color':font_color,'bg_color':background_color,'border':1,'num_format':'$0.00'})
integer_format = writer.book.add_format({'font_color':font_color,'bg_color':background_color,'border':1,'num_format':'0'})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [28]:
# writer.sheets['trades'].set_column('A:A', 18, string_format)
# writer.sheets['trades'].set_column('B:B', 18, string_format)
# writer.sheets['trades'].set_column('C:C', 18, string_format)
# writer.sheets['trades'].set_column('D:D', 18, string_format)
# writer.save()

writer.sheets['trades'].write('A1', 'Ticker', string_format)
writer.sheets['trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [31]:
column_formats = {'A':['Ticker', string_format],'B':['Stock Prices', dollar_format],'C':['Market Capitalization', dollar_format],'D':['Number of Shares to Buy',integer_format]}

for column in column_formats.keys():
    writer.sheets['trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [32]:
writer.save()

/home/i0/.pyenv/versions/3.9.1/envs/algotrading/lib/python3.9/site-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
